<a href="https://colab.research.google.com/github/osmarbraz/sri/blob/main/2_GerarPOS_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gerar POS-Tagging do dataset

Gera as POS-Tagging dos documentos do dataset.

- Utiliza os dados pré-processados do arquivo `dataset.zip` .

**Gera o arquivo `datasetpos.csv` compactado em `datasetpos.zip.zip` com com as postagging das palavras do documento.**

Cada linha do arquivo `datasetpos.csv` é formado por `["id","pos_documento"]`.
 - `"id"` é o idenficador do documento no dataset.
 - `"pos_documento"` é uma lista das sentenças do documento, formado por `"tokens","pos","verbos" e "lemma"`.
  - `"tokens"` é uma lista com os tokens da sentença.
  - `"pos"` é uma lista com as postagging das palavras da sentença.
  - `"verbos"` é uma lista com os verbos da sentença.
  - `"lemma"` é uma lista com os lemmas das palavras da sentença.

# 1 Preparação do ambiente

Preparação do ambiente para execução do script.

## 1.1 Tempo inicial de processamento

In [1]:
# Import das bibliotecas.
import time
import datetime

# Marca o tempo de início do processamento
inicio_processamento = time.time()

## 1.2 Funções e classes auxiliares

Verifica se existe o diretório do notebook no diretório corrente.   


In [2]:
# Import das bibliotecas.
import os # Biblioteca para manipular arquivos

# ============================  
def verificaDiretorioNotebook():
    """
    Verifica se existe o diretório do notebook no diretório corrente.    
    """
    
    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_NOTEBOOK):  
        # Cria o diretório
        os.makedirs(DIRETORIO_NOTEBOOK)
        logging.info("Diretório do notebook criado: {}".format(DIRETORIO_NOTEBOOK))
    
    return DIRETORIO_NOTEBOOK

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [3]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))
    
    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))    

Classe(ModelArguments) de definição dos parâmetros do modelo

In [4]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModelosParametros:
    modelo_spacy: str = field(
        default="pt_core_news_lg",
        metadata={"help": "nome do modelo do spaCy."},
    )
    sentenciar_documento: bool = field(
        default=True,
        metadata={"help": "Dividir o documento em sentenças(frases)."},
    )   

Biblioteca de limpeza de tela


In [5]:
# Import das bibliotecas.
from IPython.display import clear_output

## 1.3 Tratamento de logs

In [6]:
# Import das bibliotecas.
import logging # Biblioteca de logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s")

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## 1.4 Identificando o ambiente Colab

In [7]:
# Import das bibliotecas.
import sys # Biblioteca para acessar módulos do sistema

# Se estiver executando no Google Colaboratory
# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [8]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:root:Dispositivo GPU não encontrado


Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [9]:
# Import das bibliotecas.
import torch

def getDeviceGPU():
    """
      Retorna um dispositivo de GPU se disponível ou CPU.
    
      Retorno:
        `device` - Um device de GPU ou CPU.       
    """
        
    # Se existe GPU disponível.
    if torch.cuda.is_available():
        
        # Diz ao PyTorch para usar GPU.    
        device = torch.device("cuda")
        
        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:        
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")
        
    return device

In [10]:
# Recupera o device com GPU ou CPU
device = getDeviceGPU()

INFO:root:Sem GPU disponível, usando CPU.


Memória

Memória disponível no ambiente

In [11]:
# Importando as bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

INFO:root:Seu ambiente de execução tem  13.6 gigabytes de RAM disponível

INFO:root:Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> "Alterar tipo de tempo de execução"
INFO:root:e selecione High-RAM. Então, execute novamente está célula


## 1.6 Monta uma pasta no google drive para carregar os arquivos de dados.

In [12]:
# import necessário
from google.colab import drive

# Monta o drive na pasta especificada
drive.mount("/content/drive")     

Mounted at /content/drive


## 1.8 Instalação do spaCy

https://spacy.io/

Modelos do spaCy para português:
https://spacy.io/models/pt

In [13]:
# Instala dependências do spacy
!pip install -U pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.2 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
cvxpy 1.2.3 requires setuptools<=64.0.2, but you have setuptools 67.4.0 which is incompatible.


In [14]:
# Instala uma versão específica
!pip install -U spacy==3.4.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 2 Parametrização

## Gerais

In [15]:
# Definição dos parâmetros a serem avaliados

## Específicos

Parâmetros do modelo

In [16]:
# Definição dos parâmetros do Modelo.
model_args = ModelosParametros(     

    #modelo_spacy = "en_core_web_lg",
    #modelo_spacy = "en_core_web_md",
    #modelo_spacy = "en_core_web_sm",
    
    modelo_spacy = "pt_core_news_lg",
    #modelo_spacy = "pt_core_news_md",
    #modelo_spacy = "pt_core_news_sm",
    
    sentenciar_documento = True,
)

## Nome do diretório dos arquivos de dados

In [17]:
# Diretório do notebook
DIRETORIO_NOTEBOOK = "SRI"

## Define o caminho para os arquivos de dados

In [18]:
# Diretório local para os arquivos de dados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_NOTEBOOK + "/"

# Diretório no google drive com os arquivos de dados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/" + DIRETORIO_NOTEBOOK + "/data/"

# 3 spaCy

## 3.1 Download arquivo modelo

Uso:
https://spacy.io/usage

Modelos:
https://spacy.io/models

In [19]:
!python -m spacy download $model_args.modelo_spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-02 20:06:24.639634: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-02 20:06:24.639747: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-02 20:06:24.639766: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

## 3.2 Carrega o modelo

In [20]:
# Import das bibliotecas.
import spacy # Biblioteca do spaCy

nlp = spacy.load(model_args.modelo_spacy)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


## 3.3 Funções auxiliares spaCy

### getVerbos
Localiza os verbos da sentença

In [21]:
# Import das bibliotecas.
import spacy   
from spacy.util import filter_spans
from spacy.matcher import Matcher

# (verbo normal como auxilar ou auxilar) + vários verbos auxiliares +verbo principal ou verbo auxiliar
gramaticav1 =  [
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar                                  
                {"POS": "VERB", "OP": "?", "DEP": {"IN": ["ROOT","aux","xcomp","aux:pass"]}},  #verbo normal como auxiliar
                {"POS": "AUX", "OP": "*", "DEP": {"IN": ["aux","xcomp","aux:pass"]}},  #verbo auxiliar   
                {"POS": "VERB", "OP": "+"}, #verbo principal
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop","aux","xcomp","aux:pass"]}},  #verbo auxiliar
               ] 

# verbo auxiliar + verbo normal como auxiliar + conjunção com preposição + verbo
gramaticav2 =  [               
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar                   
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["ROOT"]}},  #verbo principal       
                {"POS": "SCONJ", "OP": "+", "DEP": {"IN": ["mark"]}}, #conjunção com preposição
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["xcomp"]}}, #verbo normal como complementar
               ] 

#Somente verbos auxiliares
gramaticav3 =  [
                {"POS": "AUX", "OP": "?"},  #Verbos auxiliar 
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop"]}},  #Verbos auxiliar de ligação (AUX+(cop))
                {"POS": "ADJ", "OP": "+", "DEP": {"IN": ["ROOT"]}}, 
                {"POS": "AUX", "OP": "?"}  #Verbos auxiliar 
               ] 

matcherv = Matcher(nlp.vocab)
         
matcherv.add("frase verbal", [gramaticav1])
matcherv.add("frase verbal", [gramaticav2])
matcherv.add("frase verbal", [gramaticav3])

#Retorna a Frase Verbal
def getVerbos(periodo):    
  #Processa o período
  doc1 = nlp(periodo.text)
  
  # Chama o mather para encontrar o padrão
  matches = matcherv(doc1)

  padrao = [doc1[start:end] for _, start, end in matches]

  #elimina as repetições e sobreposições
  #return filter_spans(padrao)
  lista1 = filter_spans(padrao)

  # Converte os itens em string
  lista2 = []
  for x in lista1:
      lista2.append(str(x))
  
  return lista2

### getTokensSentenca

Retorna a lista de tokens da sentenca.

In [22]:
def getTokensSentenca(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:    
    lista.append(token.text)

  return lista

### getListaTokensPOSSentenca

Retorna três listas uma com os tokens, as POS-Tagging e o lemma das palavras da sentenca.

In [23]:
def getListaTokensPOSSentenca(sentenca):

  """
    Retorna três listas, a primeira com os tokens da sentença, 
    a segunda com as classes moforssintáticas dos tokens e a 
    última com o lemma das palavras da sentença.

    Parâmetros:
    `sentenca` - Sentença processada pelo spaCy.
  """

  # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista_tokens = []
  lista_pos = []
  lista_lemma = []

  # Percorre a sentença adicionando os tokens e as POS
  for token in doc:    
    lista_tokens.append(token.text)
    lista_pos.append(token.pos_)
    lista_lemma.append(token.lemma_)
    
  return lista_tokens, lista_pos, lista_lemma

# 4 Gerar POS-Tagging

## 4.1 Configuração

### 4.1.1 Especifica os nomes dos arquivos de dados



In [24]:
# Nome do arquivo
NOME_ARQUIVO_DATASET = "dataset.csv"
NOME_ARQUIVO_DATASET_COMPACTADO = "dataset.zip"

### 4.1.2 Cria o diretório local para receber os dados

In [25]:
# Biblioteca para acessar o sistema de arquivos
import os

#Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):  
    # Cria o diretório
    os.makedirs(dirbase)    
    logging.info("Diretório criado: {}.".format(dirbase))
else:    
    logging.info("Diretório já existe: {}.".format(dirbase))

INFO:root:Diretório criado: /content/SRI.


## 4.2 Carrega o dataset

### 4.2.1 Copia e descompacta os arquivos do dataset do Google Drive para o Colaboratory

Copia os arquivos do google drive

In [26]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_DATASET_COMPACTADO" "$DIRETORIO_LOCAL$NOME_ARQUIVO_DATASET_COMPACTADO"

  logging.info("Terminei a cópia.")

INFO:root:Terminei a cópia.


Descompacta os arquivos.

Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens 
*   `-d` Diretório de destino



In [27]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_DATASET_COMPACTADO" -d "$DIRETORIO_LOCAL"

  logging.info("Terminei a descompactação.")

INFO:root:Terminei a descompactação.


### 4.2.2 Carregamento dos dados do dataset

In [28]:
# Import das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
df_dataset = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_DATASET, sep=";", encoding="UTF-8")

logging.info("TERMINADO ORIGINAIS: {}.".format(len(df_dataset)))

INFO:root:TERMINADO ORIGINAIS: 20.


In [29]:
df_dataset.sample(5)

,id,sentencas,documento
15,16,['O Brasil lavou a alma após o decepcionante e...,O Brasil lavou a alma após o decepcionante emp...
7,8,['A cidade de São Paulo e alguns municípios do...,A cidade de São Paulo e alguns municípios do i...
1,2,['Ao menos 17 pessoas morreram após a queda de...,Ao menos 17 pessoas morreram após a queda de u...
13,14,['A pista principal do Aeroporto Internacional...,A pista principal do Aeroporto Internacional d...
19,20,['Dois terremotos atingiram a região de Niigat...,"Dois terremotos atingiram a região de Niigata,..."


### 4.2.3 Corrigir os tipos de dados do dataset

No dataset:
- coluna 1 - `sentenças` carregadas do arquivo vem como string e não como lista.

In [30]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

# Verifica se o tipo da coluna não é list e converte
df_dataset["sentencas"] = df_dataset["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

logging.info("TERMINADO CORREÇÃO ORIGINAIS: {}.".format(len(df_dataset)))

INFO:root:TERMINADO CORREÇÃO ORIGINAIS: 20.


## 4.3 Gerar POS-Tagging Dataset

### 4.3.1 Gerar POS-Tagging



In [31]:
# Import das bibliotecas.
import re
# Biblioteca para acessar o sistema de arquivos
import os
# Biblioteca para barra de progresso
from tqdm.notebook import tqdm as tqdm_notebook

logging.info("Processando {} documentos.".format(len(df_dataset)))

# Barra de progresso dos dados
dados_bar = tqdm_notebook(df_dataset.iterrows(), desc=f"Dados", unit=f"registro", total=len(df_dataset))

# Lista para armazenar as postagging do documento
lista_dataset_pos = []

# Percorre as linhas dos documentos
for i, linha_documento in dados_bar:

    # Carrega a lista das sentenças do documento de acordo com o tipo armazenado
    lista_sentenca_documento = linha_documento[1]

    # Lista para armazenar as postagging das sentenças do documento   
    lista_sentenca_documentos_pos = []

    # Percorre as sentenças do documento
    for j, sentenca in enumerate(lista_sentenca_documento):
      #print(sentenca)

      # Processa sentença no spaCy para extrair tokens, postagging e lemma
      doc = nlp(sentenca)

      # Retorna os tokens e as postagging da sentença
      lista_tokens, lista_pos, lista_lemma = getListaTokensPOSSentenca(doc)
      
      # Retorna uma lista com os verbos da sentença
      lista_verbos = getVerbos(doc)

      # Concatena o pos do documento
      lista_sentenca_documentos_pos.append([lista_tokens, lista_pos, lista_verbos, lista_lemma])
      
    # Adiciona o documento a lista
    lista_dataset_pos.append([linha_documento[0], lista_sentenca_documentos_pos])  

INFO:root:Processando 20 documentos.


Dados:   0%|          | 0/20 [00:00<?, ?registro/s]

## 4.4 Salva as POS geradas

### 4.4.1 Especifica os nomes dos arquivos de POS do dataset


In [32]:
# Nome do arquivo
NOME_ARQUIVO_DATASET_POS = "datasetpos.csv"
NOME_ARQUIVO_DATASET_POS_COMPACTADO = "datasetpos.zip"

### 4.4.2 Gera arquivo POS

In [33]:
# Import das bibliotecas.
import pandas as pd

# Cria o dataframe da lista
df_dataset_pos = pd.DataFrame(lista_dataset_pos, columns = ["id","pos_documento"])

# Salva o arquivo das postagging
df_dataset_pos.to_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_DATASET_POS,  sep=";", index=False)

### 4.4.3 Compacta e copia a POS do dataset para uma pasta do GoogleDrive

Compacta o arquivo gerado da comparação para facilitar o envio para o GoogleDrive

In [34]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !zip -o -q -j "$DIRETORIO_LOCAL$NOME_ARQUIVO_DATASET_POS_COMPACTADO" "$DIRETORIO_LOCAL$NOME_ARQUIVO_DATASET_POS"

  logging.info("Terminei a compactação.")

INFO:root:Terminei a compactação.


Copia o arquivo para o GoogleDrive

In [35]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
   
    # Copia o arquivo das postagging       
    !cp "$DIRETORIO_LOCAL$NOME_ARQUIVO_DATASET_POS_COMPACTADO" "$DIRETORIO_DRIVE"
        
    logging.info("Terminei a cópia do arquivo.")

INFO:root:Terminei a cópia do arquivo.


### 4.4.4 Carrega os dados

Realiza um teste carregando o arquivo da POS do dataset criado.


In [36]:
# Import das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
df_dataset_pos = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_DATASET_POS, sep=";", encoding="UTF-8")

print(len(df_dataset_pos))

20


In [37]:
df_dataset_pos.sample(5)

,id,pos_documento
12,13,"[[['O', 'médico', 'pessoal', 'do', 'argentino'..."
14,15,"[[['Quase', 'metade', 'dos', 'vôos', 'previsto..."
11,12,"[[['Um', 'atirador', 'matou', 'ao', 'menos', '..."
17,18,"[[['O', 'Itaú', ',', 'segundo', 'maior', 'banc..."
9,10,"[[['Quinze', 'voluntários', 'da', 'ONG', 'fran..."


# 5 Finalização

## 5.1 Tempo final de processamento



In [38]:
# Pega o tempo atual menos o tempo do início do processamento.
final_processamento = time.time()
tempo_total_processamento = formataTempo(final_processamento - inicio_processamento)

print("")
print("  Tempo processamento:  {:} (h:mm:ss)".format(tempo_total_processamento))


  Tempo processamento:  0:02:02 (h:mm:ss)
